In [8]:
import geopandas as gpd
import pandas as pd
from shapely import wkt

Gen NTA coords

In [3]:
df = pd.read_csv('/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/generate_household_datasets/2020_Neighborhood_Tabulation_Areas__NTAs__-_Tabular_20240326.csv')
# Convert the strings to MultiPolygon objects
df['the_geom'] = df['the_geom'].apply(wkt.loads)

# Now you can get the bounds for each MultiPolygon
df['bounds'] = df['the_geom'].apply(lambda geom: geom.bounds)

# Split the bounds into separate columns
df[['min_lon', 'min_lat', 'max_lon', 'max_lat']] = pd.DataFrame(df['bounds'].tolist(), index=df.index)
df_nta_coords = df[['NTA2020', 'min_lon', 'min_lat', 'max_lon', 'max_lat']]
df_nta_coords.to_csv('/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/generate_household_datasets/nta_coords.csv', index=False)

In [13]:
import pandas as pd
import random
import folium

# Load the NTA boundary data from a CSV file
nta_data = pd.read_csv("/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/generate_household_datasets/nta_coords.csv")

# Function to generate random latitude and longitude within the NTA boundaries
def generate_coords(nta_code, num_households):
    # Filter the NTA data to get the boundaries for the specified NTA code
    nta_row = nta_data[nta_data["NTA2020"] == nta_code]

    # Check if the NTA code is valid
    if len(nta_row) == 0:
        print(f"Invalid NTA code: {nta_code}")
        return

    # Extract the minimum and maximum latitude and longitude values for the NTA
    min_lat, min_lon = nta_row["min_lat"].values[0], nta_row["min_lon"].values[0]
    max_lat, max_lon = nta_row["max_lat"].values[0], nta_row["max_lon"].values[0]

    # Generate random latitude and longitude coordinates within the NTA boundaries
    coords = []
    for _ in range(num_households):
        lat = random.uniform(min_lat, max_lat)
        lon = random.uniform(min_lon, max_lon)
        coords.append((lat, lon))

    return coords

# Example usage
nta_code = "BK0101"  # Enter the desired NTA code here
num_households = 100  # Enter the desired number of households here

coords = generate_coords(nta_code, num_households)

# Create a map centered on New York City
nyc_map = folium.Map(location=[40.7128, -74.0059], zoom_start=12)

# Add markers for the generated coordinates
if coords:
    for coord in coords:
        folium.Marker(location=coord, icon=folium.Icon(color='red')).add_to(nyc_map)

# Display the map
nyc_map